# Amateur Hour - Stock + Market News
### Starter Kernel by ``Magichanics`` 
*([Gitlab](https://gitlab.com/Magichanics) - [Kaggle](https://www.kaggle.com/magichanics))*

Stocks are unpredictable, but can sometimes follow a trend. In this notebook, we will be discovering the correlation between the stocks and the news.

If there are any things that you would like me to add or remove, feel free to comment down below. I'm mainly doing this to learn and experiment with the data. I plan on rewriting a lot of code in the future to make it look nicer, since a lot of the stuff I have written may not be the most efficient way to approach specific problems.

**What's new?**
* October 14th, 2018 - Published kernel


![title](https://upload.wikimedia.org/wikipedia/commons/8/8d/Wall_Street_sign_banner.jpg)

Source: [Wikimedia Commons](https://commons.wikimedia.org/wiki/File:Wall_Street_sign_banner.jpg)

In [1]:
import numpy as np
import pandas as pd
import os
from itertools import chain
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.linear_model import LogisticRegression
from pandas.tseries.holiday import USFederalHolidayCalendar
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
import datetime
import gc

# import environment for data
from kaggle.competitions import twosigmanews
env = twosigmanews.make_env()

Loading the data... This could take a minute.
Done!


In [2]:
sampling = True

In [16]:
(market_train_df, news_train_df) = env.get_training_data()

if sampling:
    market_train_df = market_train_df.tail(400_000)
    news_train_df = news_train_df.tail(1_000_000)
else:
    market_train_df = market_train_df.tail(3_000_000)
    news_train_df = news_train_df.tail(6_000_000) 

In [4]:
market_train_df.head()

,time,assetCode,assetName,volume,close,open,returnsClosePrevRaw1,returnsOpenPrevRaw1,returnsClosePrevMktres1,returnsOpenPrevMktres1,returnsClosePrevRaw10,returnsOpenPrevRaw10,returnsClosePrevMktres10,returnsOpenPrevMktres10,returnsOpenNextMktres10,universe
3672956,2016-02-19 22:00:00+00:00,SPWR.O,SunPower Corp,4109129.0,21.14,22.25,-0.074431,-0.094055,-0.074354,-0.081761,-0.175828,-0.102823,-0.180055,-0.111746,0.068726,1.0
3672957,2016-02-19 22:00:00+00:00,SQM.N,Sociedad Quimica y Minera de Chile SA,414021.0,17.15,16.94,0.002924,-0.033105,0.002975,-0.025269,0.051502,0.057428,0.049969,0.054736,-0.003696,0.0
3672958,2016-02-19 22:00:00+00:00,SRC.N,Spirit Realty Capital Inc,7481287.0,11.09,11.09,-0.001800,0.024954,-0.001777,0.029012,0.038390,0.052182,0.035912,0.046627,-0.067333,1.0
3672959,2016-02-19 22:00:00+00:00,SRCL.O,Stericycle Inc,898932.0,109.66,111.30,-0.016855,0.004241,-0.016824,0.008331,-0.054166,-0.052121,-0.055767,-0.054056,-0.044206,1.0
3672960,2016-02-19 22:00:00+00:00,SRE.N,Sempra Energy,2143306.0,97.25,96.66,0.003819,0.014058,0.003833,0.015573,0.020355,0.012359,0.019882,0.011141,-0.006034,1.0


In [5]:
news_train_df.head()

,time,sourceTimestamp,firstCreated,sourceId,headline,urgency,takeSequence,provider,subjects,audiences,bodySize,companyCount,headlineTag,marketCommentary,sentenceCount,wordCount,assetCodes,assetName,firstMentionSentence,relevance,sentimentClass,sentimentNegative,sentimentNeutral,sentimentPositive,sentimentWordCount,noveltyCount12H,noveltyCount24H,noveltyCount3D,noveltyCount5D,noveltyCount7D,volumeCounts12H,volumeCounts24H,volumeCounts3D,volumeCounts5D,volumeCounts7D
8328750,2015-12-08 13:56:53+00:00,2015-12-08 13:56:53+00:00,2015-12-08 13:56:53+00:00,f9c4067a6d20f21b,"CHESAPEAKE ENERGY CORP SHARES EXTEND LOSSES, N...",1,1,RTRS,"{'BLR', 'STX', 'OILG', 'EXPL', 'HOT', 'ENER', ...","{'E', 'U'}",0,1,,False,2,21,{'CHK.N'},Chesapeake Energy Corp,1,1.00000,-1,0.819143,0.125228,0.055629,21,2,3,4,7,9,17,23,24,41,63
8328751,2015-12-08 13:57:20+00:00,2015-12-08 13:57:20+00:00,2015-12-08 13:57:20+00:00,749e57557c589fca,REG - Societe Generale SA Anheuser-Busch InBev...,3,1,LSE,"{'NEWR', 'FOBE', 'WEU', 'BEVS', 'NCYC', 'LEN',...",{'LSEN'},21427,1,,False,47,1528,"{'ABI.BR', 'BUD.N'}",Anheuser Busch Inbev SA,1,1.00000,0,0.014524,0.801992,0.183484,62,0,0,0,1,3,22,30,51,90,167
8328752,2015-12-08 13:57:20+00:00,2015-12-08 13:57:19+00:00,2015-12-08 13:57:19+00:00,e61c180b2be5eb45,REG - Societe Generale SA Anheuser-Busch InBev...,3,1,LSE,"{'NEWR', 'FOBE', 'WEU', 'BEVS', 'NCYC', 'LEN',...",{'LSEN'},59958,1,,False,53,4563,"{'ABI.BR', 'BUD.N'}",Anheuser Busch Inbev SA,1,1.00000,1,0.035002,0.161918,0.803080,176,19,25,46,74,133,21,29,50,89,166
8328753,2015-12-08 13:57:37+00:00,2015-12-08 13:57:37+00:00,2015-12-08 13:57:37+00:00,35e01becdbd06d17,IIROC Trade Resumption - BIP.PR.B <BIP.N>,3,1,CNW,"{'NEWR', 'LEN', 'ELEU', 'FINS', 'US', 'DFIN', ...","{'CNR', 'CNW'}",753,1,,False,8,137,{'BIP.N'},Brookfield Infrastructure Partners LP,4,0.57735,-1,0.811987,0.129426,0.058586,87,1,1,1,1,1,1,1,1,1,10
8328754,2015-12-08 13:57:41+00:00,2015-12-08 13:57:41+00:00,2015-12-08 13:57:41+00:00,36a59986b3a81936,TRANSOCEAN'S U.S.-LISTED SHARES DOWN 2.41 PCT ...,1,1,RTRS,"{'BLR', 'STX', 'WEU', 'HOT', 'CH', 'DRIL', 'EN...","{'E', 'U'}",0,1,,False,1,14,"{'RIG.N', 'RIGN.VX', 'RIGN.BN'}",Transocean Ltd,1,1.00000,-1,0.819123,0.125241,0.055637,14,0,0,0,1,1,1,3,14,17,17


### Information on the Training Data
* There are no Unknown ``assetName`` in ``news_train_df``, but there are 24 479 rows with Unknown as the ``assetName`` in ``market_train_df``. Merging by ``assetCode`` leaves out Unknown rows, which could be problematic.
* ``Volume`` has the highest correlation in terms of ``returnsOpenNextMktres10``.
* Merging by just ``assetCodes`` greatly increases the dataframe (with just 100k rows, it has turned into 10 million rows), although merging by ``assetCodes`` and ``time`` greatly decrease the original dataframe.

### Aggregations on News Data

It helped a lot during the Home Credit competition, and in the next block of code we will be merging the news dataframe with the market dataframe. Instead of having columns with a list of numbers, we will get aggregations for each grouping. The following block creates a dictionary that will be used when merging the data.

In [6]:
news_agg_cols = [f for f in news_train_df.columns if 'novelty' in f or
                'volume' in f or
                'sentiment' in f or
                'bodySize' in f or
                'Count' in f or
                'marketCommentary' in f or
                'relevance' in f]
news_agg_dict = {}
for col in news_agg_cols:
    news_agg_dict[col] = ['mean', 'sum', 'max', 'min']
news_agg_dict['urgency'] = ['min', 'count']
news_agg_dict['takeSequence'] = ['max']

### Joining Market & News Data

The grouping method that I'll be using is from [bguberfain](https://www.kaggle.com/bguberfain), but I'll also be adding in the headlines column, as well eliminating rows that are not partnered with either the market or news data. One way I would improve this is probably group by time periods rather than exact times given in ``time`` due to the small amount of data that share the same amount of data in terms of the ``time`` column, and possibly making it a bit more efficient. Probably remove the hour/min/second?

NOTE: When you run the full dataset, expect it to take a while.

In [17]:
# update market dataframe to only contain the specific rows with matching indecies.
def check_index(index, indecies):
    if index in indecies:
        return True
    else:
        return False

# note to self: fill int/float columns with 0
def fillnulls(X):
    
    # fill headlines with the string null
    X['headline'] = X['headline'].fillna('null')

def join_market_news(market_df, news_df, nulls=False):

    print('market_df :' + str(market_df.shape))
    
    # Fix asset codes (str -> list)
    news_df['assetCodes'] = news_df['assetCodes'].str.findall(f"'([\w\./]+)'")

    # Expand assetCodes
    assetCodes_expanded = list(chain(*news_df['assetCodes']))
    assetCodes_index = news_df.index.repeat( news_df['assetCodes'].apply(len) )

    assert len(assetCodes_index) == len(assetCodes_expanded)
    df_assetCodes = pd.DataFrame({'level_0': assetCodes_index, 'assetCode': assetCodes_expanded})
    
    # get rid of any rows that will cause null values in one dataframe or the other.
    if not nulls:
        
        # gget new dataframe
        temp_news_df_expanded = pd.merge(df_assetCodes, news_df[['time', 'assetCodes']], left_on='level_0', right_index=True, suffixes=(['','_old']))
        
        # groupby dataframes
        temp_news_df = temp_news_df_expanded.copy()[['time', 'assetCode']]
        temp_market_df = market_df.copy()[['time', 'assetCode']]
        
        # get indecies on both dataframes
        temp_news_df['news_index'] = temp_news_df.index.values
        temp_market_df['market_index'] = temp_market_df.index.values
        
        # set multiindex and join the two
        temp_news_df.set_index(['time', 'assetCode'], inplace=True)
        
        # join the two
        temp_market_df_2 = temp_market_df.join(temp_news_df, on=['time', 'assetCode'])
        del temp_market_df, temp_news_df
        
        # drop nulls in any columns
        temp_market_df_2 = temp_market_df_2.dropna()
        print('dataframe relation: ' + str(temp_market_df_2.shape))
        
        # get indecies
        market_valid_indecies = temp_market_df_2['market_index'].tolist()
        news_valid_indecies = temp_market_df_2['news_index'].tolist()
        del temp_market_df_2
            
        # get index column
        market_df['market_index'] = market_df.index.values
        market_df['is_news'] = market_df['market_index'].apply(lambda x: check_index(x, market_valid_indecies))
        market_df = market_df[market_df.is_news == True]
        print('new market dataframe: ' + str(market_df.shape))
        del market_df['market_index'], market_df['is_news']
    
    # create dataframe based on groupby
    news_col = ['time', 'assetCodes', 'headline'] + sorted(list(news_agg_dict.keys()))
    news_df_expanded = pd.merge(df_assetCodes, news_df[news_col], left_on='level_0', right_index=True, suffixes=(['','_old']))
    
    # check if the columns are in the index
    if not nulls:
        news_df_expanded['news_index'] = news_df_expanded.index.values
        news_df_expanded['is_market'] = news_df_expanded['news_index'].apply(lambda x: check_index(x, news_valid_indecies))
        news_df_expanded = news_df_expanded[news_df_expanded.is_market == True]
        print('new news dataframe: ' + str(news_df_expanded.shape))
        del news_df_expanded['news_index'], news_df_expanded['is_market']

    print('creating grouped data...')

    def news_df_feats(x):
        if x.name == 'headline':
            return list(x)
    
    # groupby time and assetcode
    news_df_expanded = news_df_expanded.reset_index()
    news_groupby = news_df_expanded.groupby(['time', 'assetCode'])
    
    # get aggregated df
    news_df_aggregated = news_groupby.agg(news_agg_dict).apply(np.float32).reset_index()
    news_df_aggregated.columns = ['_'.join(col).strip() for col in news_df_aggregated.columns.values]
    
    # get any important string dataframes
    news_df_cat = news_groupby.transform(lambda x: news_df_feats(x))['headline'].to_frame()
    new_news_df = pd.concat([news_df_aggregated, news_df_cat], axis=1)
    
    # cleanup
    del news_df_aggregated
    del news_df_cat
    del news_df
    
    # rename columns
    new_news_df.rename(columns={'time_': 'time', 'assetCode_': 'assetCode'}, inplace=True)
    new_news_df.set_index(['time', 'assetCode'], inplace=True)
    
    print('merging data...')
    
    # Join with train
    market_df = market_df.join(new_news_df, on=['time', 'assetCode'])

    # cleanup
    fillnulls(market_df)
    
    print('X shape :' + str(market_df.shape))
    
    return market_df


In [18]:
%%time
X_train = join_market_news(market_train_df, news_train_df, nulls=False)

market_df :(400000, 16)
dataframe relation: (227, 4)
new market dataframe: (182, 18)
new news dataframe: (227, 30)
creating grouped data...
merging data...
X shape :(182, 104)
CPU times: user 21.8 s, sys: 632 ms, total: 22.4 s
Wall time: 22.4 s


In [19]:
X_train.head()

,time,assetCode,assetName,volume,close,open,returnsClosePrevRaw1,returnsOpenPrevRaw1,returnsClosePrevMktres1,returnsOpenPrevMktres1,returnsClosePrevRaw10,returnsOpenPrevRaw10,returnsClosePrevMktres10,returnsOpenPrevMktres10,returnsOpenNextMktres10,universe,bodySize_mean,bodySize_sum,bodySize_max,bodySize_min,companyCount_mean,companyCount_sum,companyCount_max,companyCount_min,marketCommentary_mean,marketCommentary_sum,marketCommentary_max,marketCommentary_min,sentenceCount_mean,sentenceCount_sum,sentenceCount_max,sentenceCount_min,wordCount_mean,wordCount_sum,wordCount_max,wordCount_min,relevance_mean,relevance_sum,relevance_max,relevance_min,...,noveltyCount24H_mean,noveltyCount24H_sum,noveltyCount24H_max,noveltyCount24H_min,noveltyCount3D_mean,noveltyCount3D_sum,noveltyCount3D_max,noveltyCount3D_min,noveltyCount5D_mean,noveltyCount5D_sum,noveltyCount5D_max,noveltyCount5D_min,noveltyCount7D_mean,noveltyCount7D_sum,noveltyCount7D_max,noveltyCount7D_min,volumeCounts12H_mean,volumeCounts12H_sum,volumeCounts12H_max,volumeCounts12H_min,volumeCounts24H_mean,volumeCounts24H_sum,volumeCounts24H_max,volumeCounts24H_min,volumeCounts3D_mean,volumeCounts3D_sum,volumeCounts3D_max,volumeCounts3D_min,volumeCounts5D_mean,volumeCounts5D_sum,volumeCounts5D_max,volumeCounts5D_min,volumeCounts7D_mean,volumeCounts7D_sum,volumeCounts7D_max,volumeCounts7D_min,urgency_min,urgency_count,takeSequence_max,headline
3673357,2016-02-22 22:00:00+00:00,ALV.N,Autoliv Inc,436921.0,111.96,108.99,0.029801,0.008700,0.024366,0.005473,0.126858,0.084695,NaN,NaN,-0.037786,0.0,8557.0,8557.0,8557.0,8557.0,9.0,9.0,9.0,9.0,0.0,0.0,0.0,0.0,52.0,52.0,52.0,52.0,1555.0,1555.0,1555.0,1555.0,0.036886,0.036886,0.036886,0.036886,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,3.0,3.0,3.0,3.0,17.0,17.0,17.0,17.0,3.0,1.0,1.0,[Ingram Micro Expands Availability of Acronis ...
3673877,2016-02-22 22:00:00+00:00,F.N,Ford Motor Co,33549732.0,12.56,12.24,0.038017,0.004102,0.021645,0.000050,0.096943,0.062500,0.047356,0.055504,0.027020,1.0,8557.0,8557.0,8557.0,8557.0,9.0,9.0,9.0,9.0,0.0,0.0,0.0,0.0,52.0,52.0,52.0,52.0,1555.0,1555.0,1555.0,1555.0,0.073771,0.073771,0.073771,0.073771,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,4.0,4.0,4.0,4.0,5.0,5.0,5.0,5.0,5.0,5.0,5.0,5.0,18.0,18.0,18.0,18.0,26.0,26.0,26.0,26.0,3.0,1.0,1.0,[RPT-EXCLUSIVE-Up to 90 million more Takata ai...
3674049,2016-02-22 22:00:00+00:00,HMC.N,Honda Motor Co Ltd,610950.0,26.11,26.12,0.006942,0.007716,-0.004143,0.002679,0.003845,-0.010231,-0.017810,-0.013049,0.024391,1.0,8557.0,8557.0,8557.0,8557.0,9.0,9.0,9.0,9.0,0.0,0.0,0.0,0.0,52.0,52.0,52.0,52.0,1555.0,1555.0,1555.0,1555.0,0.073771,0.073771,0.073771,0.073771,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,1.0,2.0,2.0,2.0,2.0,8.0,8.0,8.0,8.0,8.0,8.0,8.0,8.0,3.0,1.0,1.0,[RPT-EXCLUSIVE-Up to 90 million more Takata ai...
3674110,2016-02-22 22:00:00+00:00,IM.N,Ingram Micro Inc,7136909.0,35.92,36.17,-0.010741,0.004443,-0.021674,-0.002716,0.294881,0.280807,0.220617,0.269462,-0.058289,1.0,6256.0,6256.0,6256.0,6256.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,32.0,32.0,32.0,32.0,979.0,979.0,979.0,979.0,1.000000,1.000000,1.000000,1.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,36.0,36.0,36.0,36.0,51.0,51.0,51.0,51.0,3.0,1.0,1.0,[RPT-EXCLUSIVE-Up to 90 million more Takata ai...
3674877,2016-02-22 22:00:00+00:00,TM.N,Toyota Motor Corp,297787.0,106.69,106.07,0.013104,0.003690,-0.002039,-0.001471,-0.033518,-0.054971,NaN,NaN,-0.036162,0.0,8557.0,8557.0,8557.0,8557.0,9.0,9.0,9.0,9.0,0.0,0.0,0.0,0.0,52.0,52.0,52.0,52.0,1555.0,1555.0,1555.0,1555.0,0.036886,0.036886,0.036886,0.036886,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,13.0,13.0,13.0,13.0,20.0,20.0,20.0,20.0,3.0,1.0,1.0,[RPT-EXCLUSIVE-Up to 90 million more 

### Text Processing with Logistic Regression

We are going to vectorize the headlines and apply logistic regression (labels being binary as to whether the stocks go up or not). I would probably apply this same method to the universe column.

In [23]:
# reuse data
def round_scores(x):
    if x >= 0:
        return 1
    else:
        return 0

# these functions should only go towards the training data only
def get_headline_df(X_train):
    
    headlines_lst = []
    target_lst = []
    
    # iter through every headline.
    for row in range(0,len(X_train.index)):
        for sentence in X_train['headline'].iloc[row]:
            headlines_lst.append(sentence)
            target_lst.append(round_scores(X_train['returnsOpenNextMktres10'].iloc[row]))
            
    # return dataframe
    return pd.DataFrame({'headline':pd.Series(headlines_lst), 'returnsOpenNextMktres10':pd.Series(target_lst)})
    
def get_headline(headlines_df):
    
    # get headlines as list
    headlines_lst = []
    for row in range(0,len(headlines_df.index)):
        headlines_lst.append(headlines_df.iloc[row])

    # split headlines to separate words
    basicvectorizer = CountVectorizer()
    headlines_vectorized = basicvectorizer.fit_transform(headlines_lst)
    
    print(headlines_vectorized.shape)
    return headlines_vectorized, basicvectorizer

def headline_mapping(target, headlines_vectored, headline_vectorizer):
    
    print(np.asarray(target).shape)
    headline_model = LogisticRegression()
    headline_model = headline_model.fit(headlines_vectored, target)
    
    # get coefficients
    basicwords = headline_vectorizer.get_feature_names()
    basiccoeffs = headline_model.coef_.tolist()[0]
    coeff_df = pd.DataFrame({'Word' : basicwords, 
                            'Coefficient' : basiccoeffs})
    
    # convert dataframe to dictionary of coefficients
    coefficient_dict = dict(zip(coeff_df.Word, coeff_df.Coefficient))

    return coefficient_dict, coeff_df['Coefficient'].mean()

# for predictions
def get_coeff_col(X, coeff_dict, coeff_default):
    
    def get_coeff(word_lst):
        
        # iter through every word
        coeff_sum = 0
        for word in word_lst:
            if word in coeff_dict:
                coeff_sum += coeff_dict[word]
            else:
                coeff_sum += coeff_default
        
        # get average coefficient
        coeff_score = coeff_sum / len(word_lst)
        return coeff_score
        
    basicvectorizer = CountVectorizer()
    
    # loop through every item
    headlines_coeff_lst = []
    for row in range(0,len(X['headline'].index)):
        coeff_score = 0
        for i in range(0,len(X['headline'].iloc[row])):
            coeff_score += get_coeff(str(X['headline'].iloc[row][i]).split(' '))
        headlines_coeff_lst.append(coeff_score / len(X['headline'].iloc[row]))
        
    # merge coefficient frame with main
    coeff_mean_df = pd.DataFrame({'headline_coeff_mean': pd.Series(headlines_coeff_lst)})
    X = pd.concat([X.reset_index(), coeff_mean_df], axis=1)
    
    return X

In [21]:
headline_df = get_headline_df(X_train)
coefficient_dict, coefficient_default = headline_mapping(headline_df['returnsOpenNextMktres10'],
                                            *get_headline(headline_df['headline']))

(240, 871)
(240,)


In [24]:
# will be applied to X_test as well
X_train = get_coeff_col(X_train, coefficient_dict, coefficient_default)

### Extra Features ``return``

In [25]:
def extra_features(X):
    
    # Adding daily difference
    new_col = X["close"] - X["open"]
    X.insert(loc=6, column="daily_diff", value=new_col)
    X['close_to_open'] =  np.abs(X['close'] / X['open'])

In [26]:
extra_features(X_train)

### Get Time Features

This section splits the timestamp column into their own separate columns, as well as other various time features.

Possible idea: Encoding time

In [27]:
# ripped from my previous kernel, NYC Taxi Fare

# first get dates
def split_time(df):
    
    # convert to string (will find a more efficient way to do this without converting to string)
    df['time'] = df['time'].dt.strftime('%Y-%m-%d %H:%M:%S')
    
    # split date_time into categories
    df['time_day'] = df['time'].str.slice(8,10)
    df['time_month'] = df['time'].str.slice(5,7)
    df['time_year'] = df['time'].str.slice(0,4)
    df['time_hour'] = df['time'].str.slice(11,13)
    
    # source: https://www.kaggle.com/nicapotato/taxi-rides-time-analysis-and-oof-lgbm
    df['temp_time'] = df['time'].str.replace(" UTC", "")
    df['temp_time'] = pd.to_datetime(df['temp_time'], format='%Y-%m-%d %H:%M:%S')
    
    df['time_day_of_year'] = df.temp_time.dt.dayofyear
    df['time_week_of_year'] = df.temp_time.dt.weekofyear
    df["time_weekday"] = df.temp_time.dt.weekday
    df["time_quarter"] = df.temp_time.dt.quarter
    
    del df['temp_time']
    gc.collect()
    
    # convert to non-object columns
    time_feats = ['time_day', 'time_month', 'time_year', 'time_hour']
    df[time_feats] = df[time_feats].apply(pd.to_numeric)
    
    # determine whether the day is set on a holiday
    cal = USFederalHolidayCalendar()
    holidays = cal.holidays(start='2007-01-01', end='2018-09-27').to_pydatetime()
    df['on_holiday'] = df['time'].str.slice(0,10).apply(lambda x: 1 if x in holidays else 0)

In [28]:
split_time(X_train)

### Cleaning Data
Removes all categorical data as well as data that does not show up in the test data.

In [34]:
def remove_cols(X):
    del_cols = [f for f in X.columns if X[f].dtype == 'object'] + ['assetName', 'index']
    for f in del_cols:
        del X[f]

In [30]:
remove_cols(X_train)

### Compile X functions into one function

This will be used when looping through different batches of X_test

In [31]:
def get_X(market_df, news_df):
    
    # these are all the functions applied to X_train except for a few
    X_test = join_market_news(market_df, news_df, nulls=True)
    X_test = get_coeff_col(X_test, coefficient_dict, coefficient_default)
    extra_features(X_test)
    split_time(X_test)
    remove_cols(X_test)
    
    return X_test

#### Resulting Dataframe and Data Correlation to Target column
We have went to roughly 50 columns to 113!

In [32]:
X_train.head(10)

,index,volume,close,daily_diff,open,returnsClosePrevRaw1,returnsOpenPrevRaw1,returnsClosePrevMktres1,returnsOpenPrevMktres1,returnsClosePrevRaw10,returnsOpenPrevRaw10,returnsClosePrevMktres10,returnsOpenPrevMktres10,returnsOpenNextMktres10,universe,bodySize_mean,bodySize_sum,bodySize_max,bodySize_min,companyCount_mean,companyCount_sum,companyCount_max,companyCount_min,marketCommentary_mean,marketCommentary_sum,marketCommentary_max,marketCommentary_min,sentenceCount_mean,sentenceCount_sum,sentenceCount_max,sentenceCount_min,wordCount_mean,wordCount_sum,wordCount_max,wordCount_min,relevance_mean,relevance_sum,relevance_max,relevance_min,sentimentClass_mean,...,noveltyCount5D_max,noveltyCount5D_min,noveltyCount7D_mean,noveltyCount7D_sum,noveltyCount7D_max,noveltyCount7D_min,volumeCounts12H_mean,volumeCounts12H_sum,volumeCounts12H_max,volumeCounts12H_min,volumeCounts24H_mean,volumeCounts24H_sum,volumeCounts24H_max,volumeCounts24H_min,volumeCounts3D_mean,volumeCounts3D_sum,volumeCounts3D_max,volumeCounts3D_min,volumeCounts5D_mean,volumeCounts5D_sum,volumeCounts5D_max,volumeCounts5D_min,volumeCounts7D_mean,volumeCounts7D_sum,volumeCounts7D_max,volumeCounts7D_min,urgency_min,urgency_count,takeSequence_max,headline_coeff_mean,close_to_open,time_day,time_month,time_year,time_hour,time_day_of_year,time_week_of_year,time_weekday,time_quarter,on_holiday
0,3673357,436921.0,111.96,2.97,108.99,0.029801,0.008700,0.024366,0.005473,0.126858,0.084695,NaN,NaN,-0.037786,0.0,8557.0,8557.0,8557.0,8557.0,9.0,9.0,9.0,9.0,0.0,0.0,0.0,0.0,52.0,52.0,52.0,52.0,1555.0,1555.0,1555.0,1555.0,0.036886,0.036886,0.036886,0.036886,-1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,3.0,3.0,3.0,3.0,17.0,17.0,17.0,17.0,3.0,1.0,1.0,0.043021,1.027250,22,2,2016,22,53,8,0,1,0
1,3673877,33549732.0,12.56,0.32,12.24,0.038017,0.004102,0.021645,0.000050,0.096943,0.062500,0.047356,0.055504,0.027020,1.0,8557.0,8557.0,8557.0,8557.0,9.0,9.0,9.0,9.0,0.0,0.0,0.0,0.0,52.0,52.0,52.0,52.0,1555.0,1555.0,1555.0,1555.0,0.073771,0.073771,0.073771,0.073771,-1.0,...,1.0,1.0,1.0,1.0,1.0,1.0,4.0,4.0,4.0,4.0,5.0,5.0,5.0,5.0,5.0,5.0,5.0,5.0,18.0,18.0,18.0,18.0,26.0,26.0,26.0,26.0,3.0,1.0,1.0,-0.044477,1.026144,22,2,2016,22,53,8,0,1,0
2,3674049,610950.0,26.11,-0.01,26.12,0.006942,0.007716,-0.004143,0.002679,0.003845,-0.010231,-0.017810,-0.013049,0.024391,1.0,8557.0,8557.0,8557.0,8557.0,9.0,9.0,9.0,9.0,0.0,0.0,0.0,0.0,52.0,52.0,52.0,52.0,1555.0,1555.0,1555.0,1555.0,0.073771,0.073771,0.073771,0.073771,-1.0,...,1.0,1.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,1.0,2.0,2.0,2.0,2.0,8.0,8.0,8.0,8.0,8.0,8.0,8.0,8.0,3.0,1.0,1.0,-0.044477,0.999617,22,2,2016,22,53,8,0,1,0
3,3674110,7136909.0,35.92,-0.25,36.17,-0.010741,0.004443,-0.021674,-0.002716,0.294881,0.280807,0.220617,0.269462,-0.058289,1.0,6256.0,6256.0,6256.0,6256.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,32.0,32.0,32.0,32.0,979.0,979.0,979.0,979.0,1.000000,1.000000,1.000000,1.000000,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,36.0,36.0,36.0,36.0,51.0,51.0,51.0,51.0,3.0,1.0,1.0,-0.044477,0.993088,22,2,2016,22,53,8,0,1,0
4,3674877,297787.0,106.69,0.62,106.07,0.013104,0.003690,-0.002039,-0.001471,-0.033518,-0.054971,NaN,NaN,-0.036162,0.0,8557.0,8557.0,8557.0,8557.0,9.0,9.0,9.0,9.0,0.0,0.0,0.0,0.0,52.0,52.0,52.0,52.0,1555.0,1555.0,1555.0,1555.0,0.036886,0.036886,0.036886,0.036886,-1.0,...,1.0,1.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,13.0,13.0,13.0,13.0,20.0,20.0,20.0,20.0,3.0,1.0,1.0,-0.044477,1.005845,22,2,2016,22,53,8,0,1,0
5,3675542,7897799.0,7.63,0.02,7.61,-0.007802,0.010624,0.020360,-0.006027,0.123711,0.064336,0.029607,-0.010520,-0.047362,1.0,4733.0,4733.0,4733.0,4733.0,2.0,2.0,2.0,2.0,0.0,0.0,0.0,0.0,27.0,27.0,27.0,27.0,740.0,740.0,740.0,740.0,1.000000,1.000000,1.000000,1.000000,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,4.0,4.0,4.0,4.0,3.0,1.0,1.0,-0.023805,1.002628,23,2,2016,22,54,8,1,1,0
6,3675557,54746

In [33]:
X_train.corr()['returnsOpenNextMktres10'].sort_values().head(20)

wordCount_min             -0.098476
sentenceCount_min         -0.096911
urgency_min               -0.095561
bodySize_min              -0.091531
volumeCounts7D_mean       -0.078218
volumeCounts7D_max        -0.078218
volumeCounts7D_min        -0.078218
sentimentNeutral_min      -0.077455
sentimentWordCount_min    -0.076667
time_weekday              -0.073734
universe                  -0.069477
returnsClosePrevMktres1   -0.066222
companyCount_min          -0.060332
returnsClosePrevRaw1      -0.058518
companyCount_mean         -0.058301
companyCount_max          -0.057269
close_to_open             -0.057261
sentimentPositive_min     -0.056878
sentenceCount_mean        -0.055063
wordCount_mean            -0.054212
Name: returnsOpenNextMktres10, dtype: float64

### Using LGBM for Modelling

In [36]:
y_train = X_train['returnsOpenNextMktres10']
del X_train['returnsOpenNextMktres10'], X_train['universe']

In [37]:
import lightgbm as lgb
import time

# set model and parameters
params = {'learning_rate': 0.02, 
          'boosting': 'gbdt', 
          'objective': 'regression', 
          'seed': 2018}

# split data (for cross validation)
x1, x2, y1, y2 = train_test_split(X_train, 
                                  y_train, 
                                  test_size=0.25, 
                                  random_state=99)

In [38]:
# train data
t = time.time()
print('Fitting Up')

# cross validation
lgb_model = lgb.train(params, 
                        lgb.Dataset(x1, label=y1), 
                        5000, 
                        lgb.Dataset(x2, label=y2), 
                        verbose_eval=100, 
                        early_stopping_rounds=200)

print(f'Done, time = {time.time() - t}')

Fitting Up
Training until validation scores don't improve for 200 rounds.
[100]	valid_0's l2: 0.00299647
[200]	valid_0's l2: 0.00281826
[300]	valid_0's l2: 0.00278096
[400]	valid_0's l2: 0.00279217
Early stopping, best iteration is:
[262]	valid_0's l2: 0.00275734
Done, time = 0.34725499153137207


### Making Predictions

Now the difference between the training and test data would be these two columns,  ``['returnsOpenNextMktres10', 'universe']``. We will be trying to predict ``returnsOpenNextMktres10`` and using that as the ``confidenceValue``.

In [ ]:
def make_predictions(market_obs_df, news_obs_df):
    
    # predict using given model
    print(market_obs_df.shape)
    X_test = get_X(market_obs_df, news_obs_df)
    print(X_test.shape)
    prediction_values = np.clip(lgb_model.predict(X_test), -1, 1)

    return prediction_values

for (market_obs_df, news_obs_df, predictions_template_df) in env.get_prediction_days(): # Looping over days from start of 2017 to 2019-07-15
    
    print('predictions_template_df shape: ' + str(predictions_template_df.shape))
    # make predictions
    predictions_template_df['confidenceValue'] = make_predictions(market_obs_df, news_obs_df)
    
    # save predictions
    env.predict(predictions_template_df)


### Export Submission

In [ ]:
env.write_submission_file() # Writes your submission file
print('finished!')

### References:
* [Getting Started - DJ Sterling](https://www.kaggle.com/dster/two-sigma-news-official-getting-started-kernel)
* [a simple model - Bruno G. do Amaral](https://www.kaggle.com/bguberfain/a-simple-model-using-the-market-data)
* [LGBM Model - the1owl](https://www.kaggle.com/the1owl/my-two-sigma-cents-only)
* [Headline Processing - Andrew Gelé](https://www.kaggle.com/ndrewgele/omg-nlp-with-the-djia-and-reddit)